In [1]:
import time
import requests
import gspread
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from pyNBA.Data.helpers import Helpers
from pyNBA.Data.constants import CURRENT_SEASON, UNKNOWN_PLAYERS
from nba_api.stats.endpoints import CommonTeamRoster
from nba_api.stats.static.teams import find_team_by_abbreviation

import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
helpers = Helpers()

team_to_opp_team = {}
team_to_status = {}

current_player_data = pd.DataFrame(columns=[
    'TEAM', 'NAME', 'START', 'PLAYERSTATUS', 'PLAYERCHANCE'
])

URL = 'https://www.rotowire.com/basketball/nba-lineups.php'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

games = soup.find_all('div', class_='lineup is-nba')
for game in games:
    away_team = game.find('a', class_='lineup__team is-visit').find('div', class_='lineup__abbr').text
    away_team = helpers.prepare_team(away_team)
    away_lineup = game.find('ul', class_='lineup__list is-visit')
    away_player_data, away_lineup_status = helpers.get_player_data(away_lineup)
    away_player_data['TEAM'] = away_team
    away_player_data['NAME'] = away_player_data['NAME'].apply(lambda x: helpers.prepare_name(x, away_team))

    home_team = game.find('a', class_='lineup__team is-home').find('div', class_='lineup__abbr').text
    home_team = helpers.prepare_team(home_team)
    home_lineup = game.find('ul', class_='lineup__list is-home')
    home_player_data, home_lineup_status = helpers.get_player_data(home_lineup)
    home_player_data['TEAM'] = home_team
    home_player_data['NAME'] = home_player_data['NAME'].apply(lambda x: helpers.prepare_name(x, home_team))

    team_to_opp_team[away_team] = home_team
    team_to_opp_team[home_team] = away_team
    team_to_status[away_team] = away_lineup_status
    team_to_status[home_team] = home_lineup_status

    current_player_data = current_player_data.append(away_player_data)
    current_player_data = current_player_data.append(home_player_data)

current_player_data = current_player_data.loc[
    ~current_player_data['NAME'].isin(UNKNOWN_PLAYERS)
]

roster_data = pd.DataFrame()
for team_abbreviation in current_player_data['TEAM'].unique():
    print(team_abbreviation)
    team = find_team_by_abbreviation(team_abbreviation)
    team_id = team['id']

    roster = CommonTeamRoster(season=CURRENT_SEASON, team_id=team_id).get_data_frames()[0]
    time.sleep(0.500)

    roster['TEAM'] = team['abbreviation']

    roster_data = roster_data.append(roster)

roster_data = roster_data.rename(columns={'TeamID': 'TEAMID', 'PLAYER_ID': 'PLAYERID', 'PLAYER': 'NAME'})

roster_data['POSITION'] = roster_data['POSITION'].str.replace('G', 'Guard')
roster_data['POSITION'] = roster_data['POSITION'].str.replace('F', 'Forward')
roster_data['POSITION'] = roster_data['POSITION'].str.replace('C', 'Center')

current_data = roster_data.merge(current_player_data, on=['NAME', 'TEAM'], how='left')

current_data['LINEUPSTATUS'] = current_data['TEAM'].apply(lambda x: team_to_status[x])
current_data['OPP_TEAM'] = current_data['TEAM'].apply(lambda x: team_to_opp_team[x])

current_data['START'] = current_data['START'].fillna(0)
current_data['PLAYERSTATUS'] = current_data['PLAYERSTATUS'].fillna('Healthy')
current_data['PLAYERCHANCE'] = current_data['PLAYERCHANCE'].fillna(100)
current_data['SEASON'] = CURRENT_SEASON

current_data = current_data[[
    'SEASON', 'LINEUPSTATUS', 'PLAYERID', 'TEAM', 'OPP_TEAM', 'NAME', 'POSITION',
    'START', 'PLAYERSTATUS', 'PLAYERCHANCE'
    ]]

J. Green BOS
J. Jackson DET
J. Jackson MEM
B. Bogdanovic UTA
M. Morris LAC
BOS
DET
WAS
BKN
LAL
MEM
DEN
MIN
UTA
SAS
DAL
CHI
LAC
PHX
POR
GSW


In [3]:
missing_players = current_player_data.loc[
    ~current_player_data['NAME'].isin(roster_data['NAME'].unique())
    ]['NAME'].unique()

In [4]:
display(current_player_data)

,TEAM,NAME,START,PLAYERSTATUS,PLAYERCHANCE
0,BOS,Marcus Smart,1,Healthy,100.0
1,BOS,Jaylen Brown,1,Healthy,100.0
2,BOS,Jayson Tatum,1,Healthy,100.0
3,BOS,Tristan Thompson,1,Healthy,100.0
4,BOS,Daniel Theis,1,Healthy,100.0
...,...,...,...,...,...
1,GSW,Andrew Wiggins,1,Healthy,100.0
2,GSW,Kelly Oubre Jr.,1,Healthy,100.0
3,GSW,Draymond Green,1,Healthy,100.0
4,GSW,James Wiseman,1,Healthy,100.0


In [5]:
print(missing_players)

['J. Teague']


In [6]:
display(roster_data.loc[roster_data['NAME'].str.contains('Jackson')])
display(current_player_data.loc[current_player_data['NAME'].str.contains('Jackson')])

,TEAMID,SEASON,LeagueID,NAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYERID,TEAM
0,1610612765,2020,00,Frank Jackson,frank-jackson,5,Guard,6-3,205,"MAY 04, 1998",22.0,2,Duke,1628402,DET
7,1610612765,2020,00,Josh Jackson,josh-jackson,20,Guard-Forward,6-8,207,"FEB 10, 1997",23.0,3,Kansas,1628367,DET
7,1610612763,2020,00,Jaren Jackson Jr.,jaren-jackson-jr,13,Forward-Center,6-11,242,"SEP 15, 1999",21.0,2,Michigan State,1628991,MEM
0,1610612746,2020,00,Reggie Jackson,reggie-jackson,1,Guard,6-3,208,"APR 16, 1990",30.0,9,Boston College,202704,LAC


,TEAM,NAME,START,PLAYERSTATUS,PLAYERCHANCE
1,DET,Josh Jackson,1,GTD,100.0
5,MEM,Jaren Jackson Jr.,0,OUT,0.0


In [7]:
for i in missing_players:
    last = i.split()[1]
    print('------------------------------------------------------------------------------------------------------------------------')
    print(last)
    display(roster_data.loc[roster_data['NAME'].str.contains(last)])
    print('------------------------------------------------------------------------------------------------------------------------')

------------------------------------------------------------------------------------------------------------------------
Teague


,TEAMID,SEASON,LeagueID,NAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYERID,TEAM
15,1610612738,2020,00,Jeff Teague,jeff-teague,55,Guard,6-3,195,"JUN 10, 1988",32.0,11,Wake Forest,201952,BOS


------------------------------------------------------------------------------------------------------------------------


In [8]:
from pyNBA.Data.data import QueryData
query_data = QueryData(update=False)

a = query_data.query_game_data()



/Users/brandonshimiaie/Projects/pyNBA/sqlite/db/nba.db
2.6.0


In [9]:
display(a.loc[a['ID'] == '0022000073'])

,ID,SEASON,SEASONTYPE,DATE,HTM,VTM,W
7930,0022000073,2020-21,Regular Season,2021-01-01,MIL,CHI,CHI


In [10]:
from nba_api.stats.endpoints import PlayByPlayV2
play_by_play = PlayByPlayV2(game_id='0022000074').get_data_frames()[0]
display(play_by_play)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG


In [11]:
from nba_api.stats.endpoints import BoxScoreTraditionalV2
traditional_boxscores = BoxScoreTraditionalV2(
                game_id='0022000074'
                ).get_data_frames()[0]
display(traditional_boxscores)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022000074,1610612764,WAS,Washington,1630166,Deni Avdija,F,,27:14,4.0,...,0.0,7.0,7.0,3.0,3.0,0.0,1.0,3.0,11.0,32.0
1,0022000074,1610612764,WAS,Washington,1629060,Rui Hachimura,F,,19:29,3.0,...,1.0,3.0,4.0,2.0,0.0,0.0,1.0,3.0,11.0,11.0
2,0022000074,1610612764,WAS,Washington,1628418,Thomas Bryant,C,,22:01,7.0,...,2.0,5.0,7.0,2.0,0.0,0.0,2.0,1.0,18.0,25.0
3,0022000074,1610612764,WAS,Washington,203078,Bradley Beal,G,,27:58,12.0,...,0.0,3.0,3.0,7.0,1.0,0.0,2.0,1.0,31.0,23.0
4,0022000074,1610612764,WAS,Washington,203526,Raul Neto,G,,22:20,5.0,...,0.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,12.0,16.0
5,0022000074,1610612764,WAS,Washington,202722,Davis Bertans,,,19:32,4.0,...,1.0,1.0,2.0,1.0,0.0,0.0,1.0,1.0,11.0,24.0
6,0022000074,1610612764,WAS,Washington,201577,Robin Lopez,,,19:40,5.0,...,2.0,4.0,6.0,2.0,0.0,3.0,1.0,5.0,11.0,9.0
7,0022000074,1610612764,WAS,Washington,1628972,Troy Brown Jr.,,,13:51,2.0,...,0.0,3.0,3.0,1.0,1.0,0.0,0.0,1.0,4.0,5.0
8,0022000074,1610612764,WAS,Washington,202397,Ish Smith,,,20:56,2.0,...,1.0,5.0,6.0,9.0,0.0,0.0,2.0,0.0,5.0,18.0
9,0022000074,1610612764,WAS,Washington,1629010,Jerome Robinson,,,22:07,4.0,...,0.0,2.0,2.0,3.0,1.0,0.0,0.0,1.0,13.0,-5.0
